In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random 
import matplotlib as mpl
import sort_seq as ss
from sort_seq import *

from keras.backend import clear_session
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2
from keras.utils import np_utils
from keras.metrics import categorical_accuracy
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.layers import Dropout
import keras.backend
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile,\
mutual_info_classif, chi2, f_classif

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ss.set_plotting_style_2()
np.random.seed(42)


%matplotlib inline


%config InlineBackend.figure_format = 'png'

Using TensorFlow backend.


### Next steps 

add this line of code : 

X_new = SelectPercentile(f_classif, percentile=80).fit_transform(X_data, clus2)
X_new.shape

In [5]:
regulons_with_noise = pd.read_csv('~/jupyter-notebooks/rpgroup/data/ml_dfs/ecoli_ml.csv', index_col = 0)

In [6]:
regulons_with_noise.head(2)

,locustag,gene name,geneid/contrast_id,reconstructed dimension 1,reconstructed dimension 2,reconstructed dimension 3,reconstructed dimension 4,reconstructed dimension 5,reconstructed dimension 6,reconstructed dimension 7,...,cluster 1,cluster 2,cluster 3,cluster 4,cluster 5,cluster 6,cluster 7,cluster 8,cluster 9,cluster 10
16,b0019,nhaa,18,0.230012,-0.332532,-0.296651,-0.161845,-0.647282,-0.621831,0.282383,...,0,1,0,0,0,0,0,0,0,0
17,b0020,nhar,19,0.380282,-0.441964,-0.219085,-0.487243,-0.892474,-0.405036,0.454539,...,0,1,0,0,0,0,0,0,0,0


In [7]:
regulons_with_noise.loc[402][-10:]

cluster 1     1
cluster 2     1
cluster 3     1
cluster 4     1
cluster 5     0
cluster 6     0
cluster 7     0
cluster 8     0
cluster 9     1
cluster 10    0
Name: 402, dtype: object

In [8]:
X_data = regulons_with_noise.iloc[:,3:-10]
y_data = regulons_with_noise.iloc[:,-10:]

X_data.head(2)
y_data.tail()

,reconstructed dimension 1,reconstructed dimension 2,reconstructed dimension 3,reconstructed dimension 4,reconstructed dimension 5,reconstructed dimension 6,reconstructed dimension 7,reconstructed dimension 8,reconstructed dimension 9,reconstructed dimension 10,...,reconstructed dimension 4069,reconstructed dimension 4070,reconstructed dimension 4071,reconstructed dimension 4072,reconstructed dimension 4073,reconstructed dimension 4074,reconstructed dimension 4075,reconstructed dimension 4076,reconstructed dimension 4077,reconstructed dimension 4078
16,0.230012,-0.332532,-0.296651,-0.161845,-0.647282,-0.621831,0.282383,-0.373765,0.563339,-0.170241,...,-0.089639,-0.466423,-0.966650,0.737780,-0.458840,0.036191,-0.257768,-0.841564,-0.827137,1.20068
17,0.380282,-0.441964,-0.219085,-0.487243,-0.892474,-0.405036,0.454539,0.073955,0.569085,-0.099562,...,0.131865,0.601050,0.373127,0.544479,0.597727,0.891052,0.658043,-0.006034,0.348551,0.36603


,cluster 1,cluster 2,cluster 3,cluster 4,cluster 5,cluster 6,cluster 7,cluster 8,cluster 9,cluster 10
1845,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,0,0
1526,0,0,0,0,0,0,0,0,0,0
114,0,0,0,0,0,0,0,0,0,0
1675,0,0,0,0,0,0,0,0,0,0


In [9]:
#The test subset will correspond to 20% of the data at random

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42) 

In [10]:
X_train.shape

(966, 4078)

In [11]:
x_val = X_train[:300]
partial_x_train = X_train[300:]
y_val = y_train[:300]
partial_y_train = y_train[300:]

In [12]:
y_val.sum(axis = 0) / y_val.sum(axis = 0).sum() * 100

cluster 1      5.820106
cluster 2     21.693122
cluster 3     11.111111
cluster 4     13.227513
cluster 5     10.052910
cluster 6      8.465608
cluster 7      1.851852
cluster 8      1.322751
cluster 9     15.343915
cluster 10    11.111111
dtype: float64

In [13]:
partial_y_train.sum(axis = 0 )/ partial_y_train.sum(axis = 0 ).sum() *100

cluster 1      4.982206
cluster 2     20.640569
cluster 3     11.150652
cluster 4     11.387900
cluster 5     11.032028
cluster 6     10.320285
cluster 7      1.067616
cluster 8      2.491103
cluster 9     15.776987
cluster 10    11.150652
dtype: float64

In [14]:
y_test.sum(axis = 0) / y_test.sum(axis=0).sum()*100

cluster 1      6.645570
cluster 2     20.253165
cluster 3     11.075949
cluster 4     13.607595
cluster 5      9.810127
cluster 6     10.126582
cluster 7      1.898734
cluster 8      1.898734
cluster 9     12.025316
cluster 10    12.658228
dtype: float64

In [ ]:
partial_x_train.shape

In [ ]:
def train_keras_multilabel_nn(n_units=64, epochs=20, n_deep_layers=1, batch_size=128): 
    
    '''
    Trains a Keras model. 
    
    Assumes there are a X_train, y_train, x_val, y_val datasets.
    
    Returns history dataframe. 
    
    '''
    nn = Sequential()
    
    #initial layer
    nn.add(Dense(n_units, activation='relu', input_shape=(X_train.shape[1],)))
    
    #extra deep layers
    for i in range(n_deep_layers):
        nn.add(Dense(n_units, activation='relu'),
               kernel_regularizer=l2(0.001))
        nn.add(Dropout(0.25))
        
    #add final output layer
    nn.add(Dense(y_train.shape[1], activation='softmax'))
    nn.compile(optimizer='rmsprop',
              loss='binary_crossentropy', 
              metrics=['accuracy'])
    
    #print neural net architecture
    nn.summary()
    
    #fit and load history
    history = nn.fit(partial_x_train, partial_y_train, epochs=epochs,
                    batch_size= batch_size, validation_data=(x_val, y_val),
                    verbose = 0)
    
    history_df = pd.DataFrame(history.history)
    
    return nn, history_df

In [ ]:
np.where?

In [ ]:
np.where(partial_y_train.values.sum(axis = 1) == 5)

partial_x_train.iloc[162, :]

In [ ]:
X_train.shape

### Best configuration

In [ ]:
clear_session()
nn, history = train_keras_multilabel_nn(n_units= 256,
                                        epochs=8, n_deep_layers=3, batch_size=128)

In [ ]:
history

Overfits at epoch 6...

In [ ]:
nn, history = train_keras_multilabel_nn(n_units= 100, epochs=10, n_deep_layers=2, batch_size=128)

In [ ]:
history.head()

Overfits at epoch 9

In [ ]:
clear_session()
nn, history = train_keras_multilabel_nn(n_units= 40, epochs=6, n_deep_layers=1, batch_size=128)

In [ ]:
history.tail()

In [ ]:
clear_session()
nn, history = train_keras_multilabel_nn(n_units= 30,
                                        epochs=10, n_deep_layers=1, batch_size=128)

In [ ]:
history

### Load hyp TF data

In [ ]:
hyp_tf_data = pd.read_csv('../data/ml_dfs/hyp_tfs_coli_X_test.csv', index_col = 0)

hyp_tf_annot = hyp_tf_data.iloc[:, :3]

hyp_tf_annot.head(2)

hyp_tf_X = hyp_tf_data.iloc[:, 3:]

hyp_tf_X.head(2)

In [ ]:
hyp_tf_X.head(1)

### Predict data using neural network

In [ ]:
clusters_predict = nn.predict(hyp_tf_X)
np.round(clusters_predict)[:20, :]

clusters = np.argmax(np.round(clusters_predict), axis = 1)
clusters_predict.shape

In [ ]:
clusters_predict[:5, :5]

In [ ]:
np.round(clusters_predict).sum(axis = 1)

In [ ]:
clusters

In [ ]:
hyp_tf_annot['clusters'] = clusters

In [ ]:
hyp_tf_w_cluster = hyp_tf_annot[hyp_tf_annot['clusters'] != 0 ]

In [ ]:
hyp_tf_w_cluster

In [ ]:
clusters_dict = {0: 'none', 
                1: 'DNA repair',
                2: 'stress response + acid', 
                3: 'oxidative stress + drug resistance (mar-rob-sox box)', 
                4: 'fur-purr-oxyr', 
                5: 'flagella + curli + biofilm', 
                6: 'carbon metabolism (laci, rhar, melr, malt)', 
                7: 'toxin-antitoxin', 
                8: 'sugar (puur, xylr, arac, beti)', 
                9: 'purine metabolism', 
                10: 'sulfur and nitrogen metabolism'}

In [ ]:
cluster_names = [clusters_dict[val] for val in hyp_tf_w_cluster['clusters'].values]

In [ ]:
cluster_names

In [ ]:
hyp_tf_w_cluster['cluster names'] = cluster_names

In [ ]:
hyp_tf_w_cluster

In [ ]:
hyp_tf_w_cluster.to_csv('../data/ml_dfs/hyp_tfs_cluster_pred_PCA_corrected.csv')